<a href="https://colab.research.google.com/github/the-Soke/FAW-AI-BINARY-MODEL/blob/main/ONNX_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/the-Soke/FAW-AI-BINARY-MODEL.git
%cd FAW-AI-BINARY-MODEL

Cloning into 'FAW-AI-BINARY-MODEL'...
remote: Enumerating objects: 1387, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 1387 (delta 12), reused 0 (delta 0), pack-reused 1359 (from 1)
Receiving objects: 100% (1387/1387), 181.65 MiB | 18.54 MiB/s, done.
Resolving deltas: 100% (103/103), done.
/content/FAW-AI-BINARY-MODEL


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mymodel (1).pt to mymodel (1).pt


In [16]:
import json
import re

notebook_path = 'PIPELINE_METRICS.ipynb'

try:
    with open(notebook_path, 'r') as f:
        notebook_content = json.load(f)
except FileNotFoundError:
    print(f"Error: The notebook '{notebook_path}' was not found.")
    notebook_content = None

found_classes = []
if notebook_content:
    for cell in notebook_content['cells']:
        if cell['cell_type'] == 'code':
            source = "".join(cell['source'])
            for match in re.finditer(r'class\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*(\(.*?\))?\s*:', source):
                class_name = match.group(1)
                inheritance = match.group(2) if match.group(2) else ''
                found_classes.append(f"{class_name}{inheritance}")

if found_classes:
    print("Found the following class definitions in 'PIPELINE_METRICS.ipynb':")
    for cls in found_classes:
        print(cls)
    print("\nPlease identify which of these is your model class.")
else:
    print(f"No class definitions found in '{notebook_path}'.")


No class definitions found in 'PIPELINE_METRICS.ipynb'.


In [15]:
import json
import re

notebook_path = 'PIPELINE.ipynb'

try:
    with open(notebook_path, 'r') as f:
        notebook_content = json.load(f)
except FileNotFoundError:
    print(f"Error: The notebook '{notebook_path}' was not found.")
    notebook_content = None

found_classes = []
if notebook_content:
    for cell in notebook_content['cells']:
        if cell['cell_type'] == 'code':
            source = "".join(cell['source'])
            # This regex captures class names that might inherit from other classes or simple class definitions
            for match in re.finditer(r'class\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*(\(.*?\))?\s*:', source):
                class_name = match.group(1)
                inheritance = match.group(2) if match.group(2) else ''
                found_classes.append(f"{class_name}{inheritance}")

if found_classes:
    print("Found the following class definitions in 'PIPELINE.ipynb':")
    for cls in found_classes:
        print(cls)
    print("\nPlease identify which of these is your model class (e.g., FAWNet or the one associated with resnet18_faw_model).")
else:
    print(f"No class definitions found in '{notebook_path}'.")


No class definitions found in 'PIPELINE.ipynb'.


In [14]:
import json
import re

notebook_path = 'PIPELINE.ipynb'

try:
    with open(notebook_path, 'r') as f:
        notebook_content = json.load(f)
except FileNotFoundError:
    print(f"Error: The notebook '{notebook_path}' was not found.")
    notebook_content = None

if notebook_content:
    model_class_found = False
    for cell in notebook_content['cells']:
        if cell['cell_type'] == 'code':
            source = "".join(cell['source'])
            # Search for class definitions
            # This regex captures class names that might inherit from other classes (e.g., class MyModel(nn.Module):)
            # or simple class definitions (e.g., class MyModel:)
            for match in re.finditer(r'class\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*(\(.*?\))?\s*:', source):
                class_name = match.group(1)
                inheritance = match.group(2) if match.group(2) else ''
                print(f"Found class definition: {class_name}{inheritance}")
                # Heuristic to identify potential PyTorch model classes
                if "torch.nn.Module" in inheritance or "nn.Module" in inheritance or "FAWNet" in class_name:
                    print(f"Potential model class: {class_name}")
                    model_class_found = True
                elif re.search(r'super\(.*?,\s*self\)\.__init__\(\)', source): # Check for super().__init__() call
                    print(f"Potential model class (with super().__init__): {class_name}")
                    model_class_found = True

    if not model_class_found:
        print(f"No obvious model class (like FAWNet or inheriting from nn.Module) found in '{notebook_path}'. You might need to define your model architecture.")


No obvious model class (like FAWNet or inheriting from nn.Module) found in 'PIPELINE.ipynb'. You might need to define your model architecture.


In [17]:
import torch
import torch.nn as nn
from torchvision import models

# Step 1 — Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 2 — Recreate the architecture
model = models.resnet18(weights=None)  # same architecture
model.fc = nn.Linear(model.fc.in_features, 2)  # binary classifier

# Step 3 — Load the trained weights
model.load_state_dict(torch.load("model.pt", map_location=device))
model.eval()

print("✅ Model loaded and ready for export")


✅ Model loaded and ready for export


In [18]:
print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
dummy_input = torch.randn(1, 3, 224, 224, device=device)


In [21]:
!pip install onnx

onnx_path = "model.onnx"

torch.onnx.export(
    model,                            # model being run
    dummy_input,                      # model input
    onnx_path,                        # output path
    export_params=True,               # store trained weights inside model file
    opset_version=11,                 # ONNX version
    do_constant_folding=True,         # optimize constants
    input_names=['input'],            # name of the model’s input
    output_names=['output'],          # name of the model’s output
    dynamic_axes={                    # allow variable batch sizes
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print(f"✅ Model exported successfully to {onnx_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.0 MB/s eta 0:00:00


/tmp/ipython-input-2526556278.py:5: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


✅ Model exported successfully to model.onnx


In [23]:
import onnx

onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)
print("✅ ONNX model is valid and ready to use!")


✅ ONNX model is valid and ready to use!


TESTING

In [24]:
pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00


In [25]:
import onnxruntime as ort
import numpy as np

# Create a runtime session
session = ort.InferenceSession("model.onnx")

# Print input details (names, shapes, etc.)
for i in session.get_inputs():
    print(f"Input: {i.name}, shape: {i.shape}, type: {i.type}")

# Prepare dummy input (use same shape as training input, e.g. 3x224x224 for ResNet)
dummy_input = np.random.randn(1, 3, 224, 224).astype(np.float32)

# Run inference
outputs = session.run(None, {"input": dummy_input})

print("✅ Inference completed!")
print("Output shape:", outputs[0].shape)


Input: input, shape: ['batch_size', 3, 224, 224], type: tensor(float)
✅ Inference completed!
Output shape: (1, 2)


PUSHING

In [26]:
cd path/to/FAW-AI-BINARY-MODEL

[Errno 2] No such file or directory: 'path/to/FAW-AI-BINARY-MODEL'
/content/FAW-AI-BINARY-MODEL


In [27]:
!git config --global user.email "kesibosoke@gmail.com"
!git config --global user.name "the-Soke"

In [28]:
!git add .
!git commit -m "exported the ONNX file"

[main 97d915d] exported the ONNX file
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 model.onnx
 create mode 100644 model.pt


In [32]:
!git push origin main

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 79.24 MiB | 8.03 MiB/s, done.
Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/the-Soke/FAW-AI-BINARY-MODEL.git
   3d456e7..97d915d  main -> main
